In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
data1 = pd.read_excel("/content/drive/MyDrive/data/Raw_data.xlsx")
data1.head()

,Sentence #,Word,Tag
0,Sentence: 1,1)पालिकेचे,O
1,NaN,नाव,O
2,NaN,मुंबई,B-loc
3,NaN,मनपाइतर,O
4,NaN,वर्णन,O


In [ ]:
data1.count()

Sentence #     108
Word          7628
Tag           7111
dtype: int64

In [ ]:
data1.value_counts

<bound method DataFrame.value_counts of           Sentence #        Word    Tag
0        Sentence: 1  1)पालिकेचे      O
1                NaN         नाव      O
2                NaN       मुंबई  B-loc
3                NaN     मनपाइतर      O
4                NaN       वर्णन      O
...              ...         ...    ...
1032251          NaN           ;    NaN
1032252          NaN         NaN    NaN
1032253          NaN           )    NaN
1032254          NaN         NaN    NaN
1032255          NaN           )    NaN

[1032256 rows x 3 columns]>

In [ ]:
print("Number of tags: {}".format(len(data1.Tag.unique())))
frequencies = data1.Tag.value_counts()
frequencies

Number of tags: 12


O           5805
I-bname      241
B-loc        221
I-loc        194
B-cts        137
I-cts        129
B-bname      114
B-floor      109
B-unitno     107
I-floor       53
I-unitno       1
Name: Tag, dtype: int64

In [ ]:
labels_to_ids = {k: v for v, k in enumerate(data1.Tag.unique())}
ids_to_labels = {v: k for v, k in enumerate(data1.Tag.unique())}
labels_to_ids

{'O': 0,
 'B-loc': 1,
 'B-unitno': 2,
 'B-floor': 3,
 'I-floor': 4,
 'B-bname': 5,
 'I-bname': 6,
 'I-loc': 7,
 'B-cts': 8,
 'I-cts': 9,
 'I-unitno': 10,
 nan: 11}

In [ ]:

# # pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
# data = data.fillna(method='ffill')
# data.head()

NameError: ignored

In [ ]:
# let's create a new column called "sentence" which groups the words by sentence
# data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# # let's also create a new column called "word_labels" which groups the tags by sentence
# data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
# data.head()

In [ ]:
# data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
# data.head()

In [ ]:
# len(data)

In [ ]:
# data2 = data.copy()

In [ ]:
# data2

In [ ]:
# data2.to_excel("/content/drive/MyDrive/data/Raw_data2.xlsx")

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/data/Raw_data2.xlsx")
data

,Unnamed: 0,sentence,word_labels
0,0,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-f..."
1,1,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,I-loc,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-flo..."
2,2,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-flo..."
3,3,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-f..."
4,4,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-f..."
...,...,...,...
103,103,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,B-unitno,O,O,O,O,O,B-flo..."
104,104,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-f..."
105,105,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,O,B-unitno,O,O,O,O,B-flo..."
106,106,1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिक...,"O,O,B-loc,O,O,O,O,O,O,B-unitno,O,O,O,O,B-floor..."


In [ ]:
data.iloc[41].sentence

'1)पालिकेचे नाव मुंबई मनपाइतर वर्णन वर्णन सदनिका नं नं 701 701 माळा नं नं 7 वा मजला मजला इमारतीचे नाव नाव अंबर पार्क को ऑप हौ सो लि लि ब्लॉक नं नं अंधेरी पश्चिम मुंबई 400058 400058 रोड नं नं सी डी बर्फीवाला मार्ग मार्ग इतर इतर माहिती माहिती व सोबत 4 स्टील्ट मेकेनिकल कार पार्किंग( पार्किंग( ( ( ( ( ( C.T.S. Number Number Number 263 D ; ; ) ) )'

In [ ]:
data.iloc[41].word_labels

'O,O,B-loc,O,O,O,O,O,O,B-unitno,O,O,O,O,B-floor,I-floor,I-floor,O,O,O,O,B-bname,I-bname,I-bname,I-bname,I-bname,I-bname,I-bname,O,O,O,O,B-loc,I-loc,I-loc,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-cts,I-cts,O,O,O,O,O'

In [ ]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 20
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        # print(sentence)

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_split_into_words=True,
                             return_offsets_mapping=True,
                             padding='max_length',
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

In [ ]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (108, 3)
TRAIN Dataset: (86, 3)
TEST Dataset: (22, 3)


In [ ]:
training_set

In [ ]:
print(training_set[0])


{'input_ids': tensor([  101,  1015,  1007,  1328, 29876, 29870, 29877, 29851, 29854,  1327,
        29876, 29871,   100,   100,  1335, 29869, 29858, 29863,  1335, 29869,
        29858, 29863,  1335, 29869, 29858, 29863,  1335, 29869, 29858, 29863,
          100,   100,  1331, 29876, 29875, 29877, 29859, 29878,  1331, 29876,
        29875, 29877, 29859, 29878,  1314, 29867, 29848, 29868, 29857, 29878,
        29874, 29878,   100,   100,  1329, 29877, 29870, 29857, 29877, 29853,
         1327,  1011,  1338, 29878,  1011,  1015,  1321, 29878,  1314, 29863,
         1327, 29853, 29869,  1311, 29862, 29869, 29878,  1328, 29872, 29854,
        29877, 29867,   100,  1011, 20143, 22275,  1338, 29878,  1320, 29878,
         1314, 29874,  1327,  1011, 17317,  1335, 29877, 29870, 29855,  1011,
         1311, 29862, 29869, 29878,  1323, 29876, 29870, 29851, 29876,  1011,
         1311, 29862, 29869, 29878,   100,  1336, 29859, 29869,  1011,  6640,
         2581,   100,  1331, 29878, 29856, 29869, 

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
1           0
)           -100
प           -100
##ा         -100
##ल         -100
##ि         -100
##क         -100
##च         -100
न           0
##ा         -100
##व         -100
[UNK]       1
[UNK]       0
व           0
##र         -100
##ण         -100
##न         -100
व           0
##र         -100
##ण         -100
##न         -100
व           0
##र         -100
##ण         -100
##न         -100
व           0
##र         -100
##ण         -100
##न         -100
[UNK]       0
[UNK]       0
म           0
##ा         -100
##ह         -100
##ि         -100
##त         -100
##ी         -100
म           0
##ा         -100
##ह         -100
##ि         -100
##त         -100
##ी         -100
ए           5
##म         -100
##आ         -100
##य         -100
##ड         -100
##ी         -100
##स         -100
##ी         -100
[UNK]       6
[UNK]       6
ब           6
##ि         -100
##ल         -100
##ड         -100
##ि         -100
##ग         -100
न           6
-           -1

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [ ]:
training_set[2]

{'input_ids': tensor([  101,  1015,  1007,  1328, 29876, 29870, 29877, 29851, 29854,  1327,
         29876, 29871,   100,   100,  1335, 29869, 29858, 29863,  1335, 29869,
         29858, 29863,  1338, 29861, 29863, 29877, 29851, 29876,  1327,  1327,
           100,  1327,  1012,  1319, 29878,  1011, 18610,  1327,  1012,  1319,
         29878,  1011, 18610,   100,  1327,  1327,   100,  1331, 29855, 29870,
         29876,  1331, 29855, 29870, 29876,   100,  1327, 29876, 29871,  1327,
         29876, 29871,  1331, 29853, 29878, 29864, 29876,  1312, 29869, 29851,
         29857,  1312, 29869, 29851, 29857,   100,  1327,  1327,  1311, 29862,
         29869, 29878,  1328, 29872, 29854, 29877, 29867,   100, 20143, 22275,
         20143, 22275,  1333, 29879, 29857,  1333, 29879, 29857,  1333, 29879,
         29857,  1321, 29878,  1012,  1314, 29863,  1012,  1327, 29853, 29869,
          1006,  1321, 29878,  1012,  1314, 29863,  1012,  1327, 29853, 29869,
          1006,  1006,  1006,  1006,  1

In [ ]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(2.3677, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 12])

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [ ]:
from sklearn import preprocessing

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1


AttributeError: ignored